# Imports

In [ ]:
!!pip install litellm
import os
from google.colab import userdata

In [ ]:
from openai import OpenAI
from typing import List, Dict
import sys
import json
import re

# Handle Interactions

This function handles user interactions with the LLM using ChatML, with each messaging expected to have a system, user, or assistant.

In [ ]:
def generate_response(messages: List[Dict]) -> str:
   """Call LLM to get response using Deepseek."""
   # Retrieve Deepseek API key from Colab secrets and set environment variable
   deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
   client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

   response = client.chat.completions.create(
      model="deepseek-reasoner", # Specify the Deepseek model
      messages=messages,
      max_tokens=1024,
      stream=False
   )
   return response.choices[0].message.content

def list_files() -> List[str]:
    """List all files in the current directory."""
    return [f for f in os.listdir('.') if os.path.isfile(f)]

def read_file(file_name: str) -> str:
    """Read the content of a file."""
    with open(file_name, 'r') as f:
        return f.read()

In [ ]:
goals = """
You are an AI agent that provides advice about art ideation and techniques to use.

You have a similar level of knowledge and technical skill as an MFA in Fine Arts graduate.

You are specifically an expert in drawing, painting, mixed media, and using found or recycled art materials in creating art pieces, but you dabble in other physical media, such as film photography.

Your job, here, is to provide everyone from budding creatives to well-versed artists ideas for art inspiration, as well as technical help with using certain media if they inquire.
"""

actions = """
If a user makes an inquiry, extract the main points, use inductive reasoning to generalize to relevant tutorials that you can find online.

Then, use deductive reasoning to answer the question, ensuring that specific creative ideas are presented, tailored for inspiration and/or help with applying art media.

Then, cite each source using a link to the source.

Keep bullet point answers to 5 bullet points (or less) with up to 100 words that best summarize a quality answer.

Keep sentence answers to a maximum of 250 words total, no matter the complexity of the question.
"""

information = """
Be mindful of any items in the memory and make sure that the logic follows in subsequent outputs.
"""

language = """
Respond in this format:

```
Question: <question>

Bullet Point Answer:
- <bullet point 1>
- <bullet point 2>
- ...
- <bullet point n >= 5>

Resources:
- <citation 1>
- <citation 2>
- ...
- <citation n>

Paragraph Answer:
<Paragraph answer>
```
"""

In [ ]:
agent_rules = [{
    "role": "system",
    "content": f"""
{goals}

{actions}

{information}

{language}
"""}]

In [ ]:
memory = [
    {"role": "system", "content": "Keep responses within 300 words total."}
]

In [ ]:
print("Hello, I am an art inspiration agent who can help you with generating inspiration for your art or help with using art media.")

# The Agent Loop
while True:
    user_input = input("Please enter your inquiry (type 'stop' to end interaction): ")

    if user_input.lower() == "stop":
        print("Stopping agent interaction.")
        break

    # Add user's new query to memory
    memory.append({"role": "user", "content": user_input})

    # 1. Construct prompt: Combine agent rules with current memory.
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")
    print()

    # 3. The LLM's response directly adheres to the agent_rules format.

    # 4. Update memory with the LLM's response
    memory.append(
        {"role": "assistant", "content": response}
    )


Hello, I am an art inspiration agent who can help you with generating inspiration for your art or help with using art media.
Please enter your inquiry (type 'stop' to end interaction): I want ideas for drawing, with approximately one idea per week to represent spending a week on each drawing.
Agent thinking...
Agent response: ```
Question: I want ideas for drawing, with approximately one idea per week to represent spending a week on each drawing.

Bullet Point Answer:
- **Week 1: Self-Portrait Through Objects**: Draw not your face, but a carefully arranged still life of 5-10 personal items that tell your story. Focus on shading and texture.
- **Week 2: Architectural Abstraction**: Choose a local building, draw it realistically, then create a second drawing breaking its forms into geometric shapes and patterns.
- **Week 3: Botanical Blueprint**: Illustrate a plant or flower with scientific detail on one half of the page, and let the same forms morph into intricate, imaginative patterns 

## Live Agent Results

Asking the same exact questions to the Live Art Inspiration Agent yielded the following:

Agent: Hello! I am your MFA-level art guide. Ask me about ideation, techniques, or mixed media inspiration...

You: I want ideas for drawing, with approximately one idea per week to represent spending a week on each drawing.

Agent:
**Question**  
How can I structure a year of weekly drawing ideas that encourage depth and exploration, with each idea designed to occupy about a week of creative focus?

**Bullet Point Answer**  
- **Observation Week**: Draw a complex household object (like a teapot or woven basket) from three different angles, focusing on texture and light.  
- **Memory Week**: Illustrate a vivid childhood memory using only shades of a single color (e.g., sepia for nostalgia, blue for calm).  
- **Nature Week**: Create a detailed study of a plant or insect, combining scientific accuracy with artistic stylization.  
- **Abstract Emotion Week**: Express a specific emotion (like anxiety or joy) through abstract shapes, lines, and composition only.  
- **Storytelling Week**: Illustrate a key moment from a book, myth, or personal story without showing any characters—hint at them through environment and objects.  
- **Architecture Week**: Draw a building or interior space using forced perspective or an unusual viewpoint (e.g., from a mouse’s eye level).  
- **Fantasy Week**: Design an original creature by hybridizing two real animals, considering its habitat and adaptations.  
- **Seasonal Change Week**: Depict the same outdoor scene across four different seasons in one cohesive artwork.  
- **Sound Visualization Week**: Listen to a piece of instrumental music and translate its rhythm and mood into a visual pattern.  
- **Portrait with a Twist Week**: Draw a portrait where the subject’s face is partially obscured or reflected in an interesting surface (water, broken mirror, window).  
- **Cultural Symbol Week**: Research and illustrate a symbol from a culture unfamiliar to you, incorporating traditional patterns.  
- **Minimalist Week**: Reduce a complex scene (like a bustling market) to its most essential shapes and lines, using a limited palette.  

**Resources**  
- **Books**: *The Creative License* by Danny Gregory (for playful daily inspiration), *Keys to Drawing* by Bert Dodson (for technique).  
- **Websites**: Sketchbook Skool (free prompts), Line of Action (figure and object reference photos).  
- **Tools**: Try switching mediums weekly (ink, charcoal, colored pencils, digital) to explore different textures.  
- **Community**: Share progress on platforms like Instagram with hashtags like #weeklongdrawing or join Reddit’s r/learnart for feedback.  

**Paragraph Answer**  
Committing to one drawing idea per week allows you to immerse yourself in a concept without rush, fostering both technical growth and creative reflection. Each week, choose a theme that balances observation, imagination, and personal meaning—for example, starting with an object study to sharpen skills, then shifting to memory-based or abstract work to explore expression. The key is to allow yourself time to research, sketch, revise, and add details, treating each piece as a mini-project. Over a year, this practice builds a portfolio that reflects evolving style and thought. Remember, the goal isn’t perfection but consistent engagement; even “failed” drawings teach valuable lessons. Adjust themes to match your interests, and consider keeping a journal to note insights or inspirations from each week’s process.
You: I also want ideas for reusing my drawings to create sculptural pieces. Would using mod podge and acrylic medium (glossy and opaque) be a good way to go about this?

Agent:
**Question**  
How can I repurpose my weekly drawings into sculptural pieces, and are Mod Podge and acrylic mediums suitable materials for this process?  

**Bullet Point Answer**  
- **Material Suitability**: Mod Podge (matte/gloss) and acrylic mediums (glossy, opaque, gel) are excellent for collage, layering, and sealing drawings onto 3D surfaces. They provide adhesion, protection, and can add texture or transparency.  
- **Key Techniques**:  
  - **Decoupage**: Tear or cut drawings and layer them onto objects (boxes, spheres) using Mod Podge for a stained-glass or patchwork effect.  
  - **Texture Building**: Mix acrylic gel medium with drawings to create sculptural relief on canvases or armatures.  
  - **Transparency Play**: Use gloss medium to embed drawing fragments in translucent layers (like on glass or acrylic sheets).  
  - **Paper Sculpture**: Stiffen drawings with Mod Podge or matte medium, then fold, curl, or assemble into 3D forms (e.g., origami-inspired shapes).  
- **Weekly Integration**:  
  - Convert a *Portrait with a Twist* drawing into a mask by adhering it to a pre-formed base.  
  - Turn *Abstract Emotion* sketches into a hanging mobile by mounting them on lightweight frames.  
  - Use *Nature Week* drawings to cover a vase or plant pot, sealing with waterproof acrylic medium.  
- **Advantages**: Low cost, non-toxic, versatile for paper-based projects.  
- **Limitations**: Not ideal for outdoor or load-bearing pieces; may warp thin paper if not sealed evenly.  

**Resources**  
- **Supplies**: Mod Podge Dimensional Magic for raised effects; Golden Artist Colors acrylic mediums for archival quality; substrates like wooden blanks, papier-mâché forms, or thrifted objects.  
- **Tutorials**: *Dina Wakley*’s mixed-media techniques (YouTube); *The Paper Outpost* for decoupage ideas.  
- **Books**: *Paper Craft* by Dawn Dupree for structural ideas; *Mixed Media Embellishments* by MaryJo McGraw.  

**Paragraph Answer**  
Transforming your weekly drawings into sculptural pieces is a fantastic way to extend their narrative and physical presence. Mod Podge and acrylic mediums are highly suitable for this transition, acting as both adhesive and finish. For instance, after completing a *Cultural Symbol Week* drawing, you could fragment it and decoupage it onto a wooden box, using glossy medium to enhance colors and add depth. Alternatively, for *Fantasy Week*, stiffen your creature drawing with matte medium, cut it out, and suspend it within a shadow box to create a diorama. The key is to consider the substrate—seal porous surfaces like wood first, and apply drawings in layers with adequate drying time to prevent bubbling. Experiment with mixing acrylic gel medium with paper pulp from old sketches to mold custom forms. This process not only recycles your art but also encourages thinking in three dimensions, adding a tactile layer to your weekly themes. Remember to test techniques on scraps first, as paper weight and medium consistency greatly affect results.
You: Does my durable paper sculpture technique mean it's successful, and what improvements can I make?

Agent:
**Question**  
How can I evaluate the success of my durable paper sculpture technique, and what specific improvements could enhance its artistic and structural quality?  

**Bullet Point Answer**  
- **Success Indicators**:  
  - **Durability**: Sculpture resists warping, yellowing, or damage over time.  
  - **Artistic Intent**: Technique enhances (not obscures) the original drawing’s meaning.  
  - **Structural Integrity**: Pieces hold shape without sagging or collapsing.  
  - **Material Harmony**: Adhesives/finishes integrate seamlessly with paper and substrate.  
- **Improvement Strategies**:  
  - **Pre-Treat Paper**: Coat drawings with matte medium *before* cutting/forming to reduce moisture warping.  
  - **Layer Supports**: Embed lightweight armatures (wire, mesh) between paper layers for complex forms.  
  - **Experiment with Substrates**: Try mounting drawings onto rigid supports (wood panels, acrylic sheets) before sculpting.  
  - **Advanced Sealants**: Use UV-protective varnish (e.g., Golden Archival Varnish) to prevent fading if displayed in light.  
  - **Textural Variety**: Mix media by adding fibers, thin fabric, or translucent papers to contrast with your drawings.  
  - **Modular Design**: Create interlocking units from drawings to build larger, stable structures.  

**Resources**  
- **Products**: Golden Archival Varnish, Liquitex Professional Matte Medium, acid-free mounting boards.  
- **Tools**: Bone folder for crisp folds, laser cutter (if accessible) for precise paper engineering.  
- **Inspiration**: Artists like *Jennifer Collier* (paper textiles), *Élise Morin* (large-scale paper installations).  
- **Testing**: Keep a sample journal with technique notes, exposure to light/humidity, and aging observations.  

**Paragraph Answer**  
Your technique’s success depends on both longevity and artistic impact. If your sculptures remain stable and visually compelling, you’ve achieved a functional foundation. To refine further, consider **pre-emptive stabilization**—brush a thin layer of matte medium onto drawings before sculpting to minimize water-based adhesive effects. For structural upgrades, integrate **hidden supports**: wrap wireframes with paper strips from discarded sketches, or use corrugated cardboard as a lightweight core. Experiment with **substrate hybrids**, like adhering drawings to curved wooden forms or acrylic domes, to challenge traditional paper’s flatness. Artistically, push contrast by pairing opaque acrylic medium areas with translucent rice paper overlays from your drawing fragments. Finally, document each sculpture’s response to environmental factors—this turns your weekly practice into a research-driven exploration of materiality, ensuring continuous improvement anchored in observation.